In [4]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from numpy import dot
from numpy.linalg import norm
import itertools

## Read SimLex

In [9]:
simlex = pd.read_csv('./data/MSimLex999_Polish.txt', sep='\t', header=None)

In [10]:
simlex.columns = ['id', 'word1', 'word2', 'similarity', 'relatedness']

In [11]:
simlex.head()

,id,word1,word2,similarity,relatedness
0,1,stary,nowy,0.43,7.29
1,2,bystry,inteligentny,8.86,9.71
2,3,ciężki,trudny,4.86,7.29
3,4,szczęśliwy,radosny,8.14,8.86
4,5,łatwy,męczący,0.43,6.43


In [12]:
simlex.describe()

,id,similarity,relatedness
count,999.000000,999.000000,999.000000
mean,500.000000,2.476697,5.945395
std,288.530761,2.553304,2.531229
min,1.000000,0.000000,0.000000
25%,250.500000,0.430000,4.430000
50%,500.000000,1.570000,6.570000
75%,749.500000,4.000000,7.860000
max,999.000000,9.710000,9.860000


In [13]:
unique_words = set(list(simlex['word1'].unique()) + list(simlex['word2'].unique()))

In [14]:
len(unique_words)

1139

## Read embeddings

In [15]:
embeddings = {}

with open("./data/kgr10.plain.lemma.cbow.dim100.neg10.vec", "r") as vec:
    next(vec)
    for line in tqdm(vec):
        line = line.split()
        if line[0] in unique_words:
            embeddings[line[0]] = np.array(line[1:]).astype(float)

2137684it [00:25, 84830.05it/s]


In [16]:
len(embeddings)

1117

### Missing embeddings

In [14]:
found_embeddings = set(embeddings.keys())

In [15]:
unique_words - found_embeddings

{'byk',
 'cela',
 'duma',
 'dąb ',
 'furia',
 'głupi ',
 'kostka',
 'księżyc',
 'ludzie',
 'luka',
 'meble',
 'mężczyźni',
 'nasiona',
 'obładowany',
 'partia',
 'pieniądze',
 'przestraszony',
 'rak',
 'ruch',
 'sława',
 'ubrania',
 'łódź'}

In [18]:
filtered_simlex = simlex[(simlex['word1'].isin(found_embeddings)) & (simlex['word2'].isin(found_embeddings))].reset_index(inplace=False)

In [20]:
print(len(simlex))
print(len(filtered_simlex))

999
967


## Similarity / relatedness metrics

In [48]:
def cosinus(w1, w2):
    v1 = embeddings[w1]
    v2 = embeddings[w2]

    return dot(v1, v2)/(norm(v1) * (norm(v2)))

In [49]:
def add_cosinus(simlex_row):
   
    simlex_row['cosinus'] = cosinus(
        simlex_row['word1'], simlex_row['word2']
    )

    return simlex_row

In [50]:
def euclidean(w1, w2):
    v1 = embeddings[w1]
    v2 = embeddings[w2]

    return np.sqrt(np.sum((v1 - v2)**2))

In [51]:
def add_euclidean(simlex_row):
    
    simlex_row['euclidean'] = euclidean(
        simlex_row['word1'], simlex_row['word2']
    )

    return simlex_row

In [23]:
tqdm.pandas()

In [24]:
filtered_simlex = filtered_simlex.progress_apply(add_cosinus, axis=1)

100%|██████████| 967/967 [00:00<00:00, 1175.05it/s]


In [25]:
filtered_simlex = filtered_simlex.progress_apply(add_euclidean, axis=1)

100%|██████████| 967/967 [00:00<00:00, 1234.41it/s]


In [26]:
filtered_simlex.tail()

,index,id,word1,word2,similarity,relatedness,cosinus,euclidean
962,994,995,dołączyć,zdobyć,0.43,2.29,0.657871,25.897192
963,995,996,wysyłać,uczestniczyć,0.00,0.86,0.560702,29.530632
964,996,997,zbierać,uczestniczyć,0.00,0.71,0.617206,26.957282
965,997,998,pochłonąć,wycofać,0.00,0.57,0.720902,21.570204
966,998,999,uczestniczyć,przybyć,0.57,3.43,0.727095,22.198529


In [27]:
filtered_simlex.to_csv('out/embeddings_results.csv')

## K top similar words

In [112]:
def k_top(k, words, metric_fun, max=True):
    
    combs = list(itertools.combinations(words, 2))
    metrics = {}
    for comb in tqdm(combs):
        metric = metric_fun(comb[0], comb[1])
        metrics[str(set(comb))] = metric

    results = {} # comb: metric

    combs = list(itertools.combinations(words, k))
    for comb in combs:
        sub_combs = list(itertools.combinations(comb, 2))
        metrics_to_mean = []
        for sub_comb in sub_combs:
            metrics_to_mean.append(
                metrics[str(set(sub_comb))]
            )
        results[str(comb)] = np.mean(np.array(metrics_to_mean))

    results = {k: v for k, v in sorted(results.items(), key=lambda item: item[1], reverse=max)}

    print()
    print()
    print('TOP 1')
    print('Słowa: ', list(results.items())[0][0])
    print('Wartość metryki: ', round(list(results.items())[0][1], 4))
    print()
    print('TOP 2')
    print('Słowa: ', list(results.items())[1][0])
    print('Wartość metryki: ', round(list(results.items())[1][1], 4))
    print()
    print('TOP 3')
    print('Słowa: ', list(results.items())[2][0])
    print('Wartość metryki: ', round(list(results.items())[2][1], 4))

In [113]:
words = [
    'sufit',
    'pochłonąć',
    'okoliczność',
    'rubież',
    'upraszać',
    'rytm',
    'nowoczesny',
    'pojemnik',
    'gwałtowny',
    'pudełko'
]

### Cosinus

In [114]:
k_top(2, words, cosinus, max=True)

100%|██████████| 45/45 [00:00<00:00, 9331.27it/s]

TOP 1
Słowa:  ('pojemnik', 'pudełko')
Wartość metryki:  0.6545

TOP 2
Słowa:  ('pochłonąć', 'gwałtowny')
Wartość metryki:  0.652

TOP 3
Słowa:  ('okoliczność', 'gwałtowny')
Wartość metryki:  0.5974



In [115]:
k_top(3, words, cosinus, max=True)

100%|██████████| 45/45 [00:00<00:00, 29997.41it/s]

TOP 1
Słowa:  ('pochłonąć', 'upraszać', 'gwałtowny')
Wartość metryki:  0.5812

TOP 2
Słowa:  ('pochłonąć', 'nowoczesny', 'gwałtowny')
Wartość metryki:  0.5589

TOP 3
Słowa:  ('pochłonąć', 'okoliczność', 'gwałtowny')
Wartość metryki:  0.5588



### Euclidean

In [116]:
k_top(2, words, euclidean, max=False)

100%|██████████| 45/45 [00:00<00:00, 31115.01it/s]

TOP 1
Słowa:  ('pochłonąć', 'gwałtowny')
Wartość metryki:  21.4629

TOP 2
Słowa:  ('pojemnik', 'pudełko')
Wartość metryki:  21.7708

TOP 3
Słowa:  ('pochłonąć', 'upraszać')
Wartość metryki:  23.3221



In [117]:
k_top(3, words, euclidean, max=False)

100%|██████████| 45/45 [00:00<00:00, 33037.58it/s]

TOP 1
Słowa:  ('pochłonąć', 'upraszać', 'gwałtowny')
Wartość metryki:  23.5361

TOP 2
Słowa:  ('pochłonąć', 'rubież', 'gwałtowny')
Wartość metryki:  24.1262

TOP 3
Słowa:  ('pochłonąć', 'okoliczność', 'gwałtowny')
Wartość metryki:  24.159

